In [ ]:
"""   

Copyright 2016, The Authors.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
   
"""

'   \nCopyright 2016, The Authors.\n\nLicensed under the Apache License, Version 2.0 (the "License");\nyou may not use this file except in compliance with the License.\nYou may obtain a copy of the License at\n\n    http://www.apache.org/licenses/LICENSE-2.0\n\nUnless required by applicable law or agreed to in writing, software\ndistributed under the License is distributed on an "AS IS" BASIS,\nWITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\nSee the License for the specific language governing permissions and\nlimitations under the License.\n   \n'

# Create Labeled Dataset

---
This notebook runs a frozen inference on a dataset.  
  
Input:


*   directory containing original images
*   frozen_inference_graph.pb  

Output:


*   directory containing labeled images







# Libraries

In [ ]:
from pathlib import Path
import os
from PIL import Image
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
from skimage.transform import resize
from google.colab.patches import cv2_imshow
import cv2

TensorFlow 1.x selected.


# Input Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Add custom input data

In [ ]:
# Add the path to the directory containing original images to PATH_TO_ORIGNALIMAGES = "..."

PATH_TO_ORIGNALIMAGES="/content/drive/MyDrive/Master's thesis/Original_Images"

In [ ]:
# Create new "Output" folder containing labeled images in the parent directory.
PARENT_DIR = str(Path(PATH_TO_ORIGNALIMAGES).parent)
OUTPUT_PATH = os.path.join(PARENT_DIR, "Labeled_Images")
print(OUTPUT_PATH)

try:
    os.mkdir(OUTPUT_PATH) 
except:
    print("older already exists")

/content/drive/MyDrive/Master's thesis/Labeled_Images
older already exists


In [ ]:
# upload exported frozen_inference_graph.pb into current working directory
!pwd
from google.colab import files
uploaded = files.upload()


/content


Saving frozen_inference_graph.pb to frozen_inference_graph.pb


# Run inferences

In [ ]:
class DeepLabModel():
    """Class to load deeplab model and run inference."""

    INPUT_TENSOR_NAME = 'ImageTensor:0'
    OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
    #INPUT_SIZE = 1921


    def __init__(self, path):
        """Creates and loads pretrained deeplab model."""
        self.graph = tf.Graph()

        graph_def = None
        # Extract frozen graph from tar archive.

        with tf.gfile.GFile(path, 'rb')as file_handle:
            graph_def = tf.GraphDef.FromString(file_handle.read())

        if graph_def is None:
            raise RuntimeError('Cannot find inference graph')

        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')

        self.sess = tf.Session(graph=self.graph)


    def run(self, image):
        """Runs inference on a single image.

        Args:
          image: A PIL.Image object, raw input image.

        Returns:
          seg_map: np.array. values of pixels are classes
        """

        width, height = image.size

        resize_ratio = 1.0 #* self.INPUT_SIZE / max(width, height)
        target_size = (int(resize_ratio * width), int(resize_ratio * height))

        resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)

        batch_seg_map = self.sess.run(
           self.OUTPUT_TENSOR_NAME,
           feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})

        seg_map = batch_seg_map[0]        

        seg_map = resize(seg_map.astype(np.uint8), (height, width), preserve_range=True, order=0, anti_aliasing=False)


        return seg_map

In [ ]:
model = DeepLabModel("frozen_inference_graph.pb")

# as may colors as you have classes

# Material color:

#colors = [(0,   0,   0),(255,  255, 0), (240,  0, 240),(0,  240, 100),(0,  210, 255)] 
#classes=5


# Damage Color:
colors = [(0,   0,   0),(0,  0, 240)] 
classes = 2

In [ ]:
for filename in os.listdir(PATH_TO_ORIGNALIMAGES):
    img = Image.open(os.path.join(PATH_TO_ORIGNALIMAGES,filename))
    seg_map = model.run(img)
    img_arr = np.array(img)
    for c in range(0, classes):
        img_arr[seg_map == c] = 0.5 * img_arr[seg_map == c] + 0.5 * np.array(colors[c])
    cv2.imwrite(os.path.join(OUTPUT_PATH,filename), img_arr)